## 💸 Stock Trend Analysis Framework

### 📥 Setups

Installation and Import of required packages

In [1147]:
# !pip install -r requirements.txt

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

Initialization of Analysis Parameters

In [1149]:
TICKERS: list[str] = ["AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "JNJ", "JPM", "XOM", "PG", "WMT", "AMT"]
START_DATE: str = "2022-01-01"
END_DATE: str = "2025-07-18"

### 🏗️ Data Acquisition

Function for Fetching Daily Price Data

In [1150]:
def get_stock_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames with date-indexed adjusted closing price data
    """
    
    data = {}

    for ticker in tickers:
        try:
            # Fetch data from Yahoo Finance and select relevant columns
            df = yf.download(ticker, start=start, end=end, auto_adjust=True)[['Close', 'Volume']]
            df.columns = ['Close', 'Volume']
            
            if not df.empty:
                data[ticker] = df
            else:
                print(f"No data found for {ticker} in the specified date range.")
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data

### 🧹 Data Cleaning and Preparation

Function for Filling Missing Values and Normalizing Values

In [1151]:
def clean_data(data: dict[str, pd.DataFrame]) -> dict:
    """
    Cleans and prepares the stock data for analysis.
    
    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by ticker symbols

    Returns:
        dict[str, pd.DataFrame]: Dictionary of DataFrames with cleaned and normalized data
    """

    cleaned_data = {}

    for ticker, df in data.items():
        # Replace zeros in 'Close' with NaN
        df.loc[df['Close'] == 0, 'Close'] = np.nan

        # Forward and backward fill missing values
        df = df.ffill()
        
        # Normalize the 'Close' prices to a range of 0 to 1
        df['Close_Normalized'] = (df['Close'] - df['Close'].min()) / (df['Close'].max() - df['Close'].min())

        df = df[['Close', 'Close_Normalized', 'Volume']] # Change column order
        
        cleaned_data[ticker] = df

    return cleaned_data

### 📊 Data Analysis

Functions for Calculating Return, Moving Average, and Volatility

In [1152]:
def calculate_return(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the daily return of the stock based on the 'Close' price.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices

    Returns:
        pd.DataFrame: DataFrame with an additional 'Return' column
    """
    
    df['Return'] = df['Close'].pct_change()
    return df

def calculate_moving_average(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the moving average of the 'Close' price over a specified window.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        window (int): The number of periods over which to calculate the moving average
    
    Returns:
        pd.DataFrame: DataFrame with an additional column for the moving average
    """

    df['Moving_Avg'] = df['Close'].rolling(window=window).mean()
    return df

def calculate_volatility(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the volatility of the stock based on the 'Return' column over a specified window.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Return' column
        window (int): The number of periods over which to calculate the volatility
    
    Returns:
        pd.DataFrame: DataFrame with an additional column for the volatility
    """

    df['Rolling_Vol'] = df['Return'].rolling(window=window).std()
    return df

Function for Calculating all the Measures

In [1153]:
def calculate_measures(data: dict[str, pd.DataFrame], ma_window: int = 50, vol_window: int = 20) -> dict:
    """
    Calculates the return, moving average and the volatility for each stock's closing price.
    
    Parameters:
        data (dict[str, pd.DataFrame]): Dictionary of DataFrames indexed by ticker symbols
        ma_window (int): Window size for moving average
        vol_window (int): Window size for rolling volatility calculation

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames with volatility added
    """
    
    final_data = {}

    for ticker, df in data.items():
        df = calculate_return(df)
        df = calculate_moving_average(df, ma_window)
        df = calculate_volatility(df, vol_window)

        final_data[ticker] = {
            'data': df,
            'overall_vol': float(df['Return'].std())
        }

    return final_data

Functions for Calculating RSI and MACD

In [1154]:
def calculate_RSI(df: pd.DataFrame, window: int) -> pd.DataFrame:
    """
    Calculates the Relative Strength Index (RSI) for a given DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        period (int): The number of periods to use for RSI calculation

    Returns:
        pd.DataFrame: DataFrame with an additional 'RSI' column
    """

    df = df.copy()

    avg_gain = (df['Return'].where(df['Return'] > 0, 0)).rolling(window=window).mean()
    avg_loss = (-df['Return'].where(df['Return'] < 0, 0)).rolling(window=window).mean()
    
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    return df

In [1155]:
def calculate_MACD(df: pd.DataFrame, short_window: int, long_window: int, signal_window: int) -> pd.DataFrame:
    """
    Calculates the Moving Average Convergence Divergence (MACD) for a given DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with 'Close' prices
        short_window (int): Short-term EMA window
        long_window (int): Long-term EMA window
        signal_window (int): Signal line EMA window

    Returns:
        pd.DataFrame: DataFrame with additional EMA, MACD, MACD Signal, and MACD Histogram columns
    """

    df = df.copy()

    df['EMA_short'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['EMA_long'] = df['Close'].ewm(span=long_window, adjust=False).mean()

    df['MACD'] = df['EMA_short'] - df['EMA_long']
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_window, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    
    return df

Function for Calculating Both Indicators

In [1156]:
def calculate_indicators(data: dict[str, dict[str, pd.DataFrame | float]], rsi_window: int = 14, short_window: int = 12, long_window: int = 26, signal_window: int = 9):
    """
    Calculates the RSI and MACD for given DataFrames.

    Parameters:
        data (dict[str, dict[str, pd.DataFrame | float]]): Dictionary of DataFrames indexed by ticker symbols
        rsi_period (int): The number of periods to use for RSI calculation (Default to 14)
        short_window (int): Short-term EMA window (Default to 12)
        long_window (int): Long-term EMA window (Default to 26)
        signal_window (int): Signal line EMA window (Default to 9)

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames with EMA, MACD, MACD Signal, and MACD Histogram added
    """

    final_data = {}

    for ticker, d in data.items():
        df = d['data']

        df = calculate_RSI(df, rsi_window)
        df = calculate_MACD(df, short_window, long_window, signal_window)

        final_data[ticker] = {
            'data': df,
            'overall_vol': d['overall_vol']
        }

    return final_data

Function for Filtering Missing Values after all Calculations

In [ ]:
def filter_data(data: dict[str, dict[str, pd.DataFrame | float]]):
    '''
    Filters out rows of missing values for given dataFrames

    Parameters:
        data (dict[str, dict[str, pd.DataFrame | float]]): Dictionary of DataFrames indexed by ticker symbols

    Returns:
        dict[str, dict[str, pd.DataFrame | float]]: Dictionary of DataFrames without NaNs
    '''

    final_data = {}

    for ticker, d in data.items():
        df = d['data']

        df = df.dropna(subset = ['Return', 'Moving_Avg', 'Rolling_Vol', 'RSI'])

        final_data[ticker] = {
            'data': df,
            'overall_vol': d['overall_vol']
        }
    return final_data

### 📈 Visualization